## EXPLORING IMAGE EXTRACTION USING MOBILENETV1
jadi disini bakal coba ekstrak kolom image_url dari Cleaned_DropNull.csv dataset  

In [2]:
import pandas as pd

# Tentukan path ke file dataset Anda
file_path = r'E:\$7th\TA\Eksploring_TF-IDF\DATA\Cleaned_DropNull.csv'

# Muat dataset ke dalam DataFrame pandas
try:
    df = pd.read_csv(file_path)
    print("Dataset berhasil dimuat.")
    print("-" * 30)
    
    # Cek ringkasan dataset
    total_rows = len(df)
    image_url_count = df['image_url'].notna().sum()
    unique_image_url_count = df['image_url'].nunique()

    print(f"Total baris dalam dataset: {total_rows}")
    print(f"Jumlah baris dengan image_url: {image_url_count}")
    print(f"Jumlah image_url unik: {unique_image_url_count}")
    
except FileNotFoundError:
    print(f"ERROR: File tidak ditemukan di path '{file_path}'. Mohon periksa kembali path file Anda.")

Dataset berhasil dimuat.
------------------------------
Total baris dalam dataset: 16526
Jumlah baris dengan image_url: 16526
Jumlah image_url unik: 16208


### DOWNLOAD GAMBAR
nanti download gambar dari image_url

In [ ]:
import os
import requests
import pandas as pd
from tqdm import tqdm

# Pastikan Anda sudah menjalankan cell pertama dan memiliki DataFrame 'df'

# --- Setup Folder Penyimpanan ---
# Tentukan path folder untuk menyimpan gambar
save_dir = r'E:\$7th\TA\Eksploring_TF-IDF\DATA\image_url'

# Buat folder jika belum ada, exist_ok=True berarti tidak akan error jika folder sudah ada
os.makedirs(save_dir, exist_ok=True)
print(f"Gambar akan disimpan di folder: {save_dir}")


# --- Fungsi untuk Mengunduh Satu Gambar ---
def download_image(row):
    # Ambil URL dari baris saat ini
    url = row['image_url']
    
    # Periksa apakah URL valid
    if not isinstance(url, str) or not url.startswith('http'):
        return 'invalid_url'
        
    try:
        # Buat nama file unik berdasarkan indeks baris DataFrame
        # .name akan mengambil nilai index saat menggunakan .apply()
        filename = f"{row.name}.jpg"
        save_path = os.path.join(save_dir, filename)
        
        # Lakukan request untuk mengunduh gambar, timeout 15 detik
        response = requests.get(url, stream=True, timeout=15)
        
        # Jika request berhasil (kode status 200)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            return save_path # Kembalikan path file lokal jika berhasil
        else:
            return f'http_error_{response.status_code}'
            
    except requests.exceptions.RequestException as e:
        # Tangani error koneksi, timeout, dll.
        return 'download_failed'

# --- Terapkan Fungsi Download ke Setiap Baris ---
# Inisialisasi tqdm untuk pandas
tqdm.pandas(desc="Downloading Images")

# Terapkan fungsi download_image ke setiap baris di DataFrame (axis=1)
# Hasilnya (path file lokal atau status error) akan disimpan di kolom baru 'local_path'
df['local_path'] = df.progress_apply(download_image, axis=1)


# --- Tampilkan Ringkasan Hasil Download ---
print("\nProses pengunduhan selesai.")
print("-" * 30)

# Hitung jumlah gambar yang berhasil diunduh
# (yaitu yang path-nya diawali dengan save_dir dan bukan status error)
success_count = df['local_path'].str.startswith(save_dir, na=False).sum()
total_images = df['image_url'].notna().sum()

print(f"Total URL gambar yang valid: {total_images}")
print(f"Berhasil diunduh: {success_count}")
print(f"Gagal diunduh: {total_images - success_count}")

print("\n5 baris pertama data dengan path gambar lokal:")
print(df.head())

Gambar akan disimpan di folder: E:\$7th\TA\Eksploring_TF-IDF\DATA\image_url
